#### 一連の手続き.ipynbでやってきた処理をひとまとめにする。考慮すべき事項は下につけ足していくことにする。

In [103]:
#必要なモジュールのimport 
import astropy.io.fits as fits
import numpy as np
import glob
import os
from scipy.ndimage import gaussian_filter
import pandas as pd
import shutil


def readheader(filename):
    return fits.open(filename)[0].header


def readdata(filename):
    return fits.open(filename)[0].data


ほかのファイルでダークのコマンドリストを作成する。
そのコマンドから読み取っていくことにする。

In [104]:
#天体画像を一つ指定したとする　→　obsfile
#darkのcsvファイルから、天体の画像の、日付、PIXTIM, RRSTRT, YSTRTが等しいものを取得する
#darklist = pd.read_csv(filename,header, skiprows)

#def judge(information):
#   darklist[information] == obsfileheader[information]
#judge("DATE-OBS") & judge("Q_PIXTIM") & judge("Q_RRSTRT") & judge("YSTRT")
# filepath = os.path.join()

In [105]:
obsfilename = #入力
dark1 = #入力
dark2 = #入力
obs_comq = #入力

SyntaxError: invalid syntax (4072968027.py, line 1)

### darkが欲しいものと一致しているとして処理をする

In [106]:
#関数の定義
def readheader(filename):
    return fits.open(filename)[0].header

def readdata(filename):
    return fits.open(filename)[0].data

def q_bsep_posi(input):
    return input[:,::2,:,:]
def q_bsep_nega(input):
    return input[:,1::2,:,:]   
#z方向平均
def mean_z(input):
    return np.mean(input, axis=1, keepdims=True)

#gaussian filter


def gaussfilter(input):
    output = gaussian_filter(input[0, 0, :, :], sigma=6, order=0,
                             output=None, mode='nearest', cval=0.0, truncate=4.0)
    return output


def q_subch(input):
    #z方向については、COMQは1となっているはずである。
    input_dim2 = input[0, 0, :, :]
    #x軸方向に20*240と分割し、z方向に積み上げる

    divstack = np.stack(np.split(input_dim2, 16, axis=1))
    #z方向にmedianをとり、20*240をtileして320*240にする
    beforetile = np.median(divstack, axis=0)  # keepdimsは不要(二次元でいい)
    tilemedian = np.tile(beforetile, 16)
    # print(tilemedian.shape)
    input_to_output = input.copy()
    input_to_output[0, 0, :, :] -= tilemedian
    # input_to_output[0, 0, :, :] = 1
    return input_to_output

#fitsファイル作成用

def makefits(dataname, filename):
    hdu = fits.PrimaryHDU(data=dataname)
    fits.HDUList([hdu]).writeto(str(filename), overwrite=True)


In [107]:
# darkheader1 = readheader(dark1)
# darkheader2 = readheader(dark2)
# obsheader = readheader(obsfilename)
# print(darkheader1["Q_CHEB"] == darkheader2["Q_CHEB"])
# print()
def process(obsfilename):
    
    """  
    ここには天体画像から、ダークを特定する処理を記載する。
    
    """
    if (darkheader1["Q_CHAM"] != 1) or (obsheader["Q_CHAM"]!=1):
        print("CoAdd Modeが1ではない。")
    elif darkheader1["Q_CHAM"]==1 and darkheader2["Q_CHAM"]==1:
        darkfile1 = readdata(dark1)
        darkfile2 = readdata(dark2)
        meandark = (np.mean(dark1, axis=1, keepdims=True)+np.mean(dark2, axis=1, keepdims=True)) /2
        meandark_1 = meandark / int(darkheader1["Q_CHEB"])

        #darkの差し引き
        #Q_CHEB倍する
        obsdata = readdata(obsfilename)
        meandark_obs = meandark_1 * obsheader["Q_CHEB"]
        
        skydata = obsdata - meandark_obs
        skydata_p = q_bsep_posi(skydata)
        skydata_n = q_bsep_nega(skydata)
        
        #z方向平均
        sky_pa = mean_z(skydata_p)
        sky_na = mean_z(skydata_n)
        
        #gaussian　平均
        sky_paG = gaussfilter(sky_pa)
        sky_naG = gaussfilter(sky_na)
        
        #Flatの作成
        sky_paF = sky_pa / sky_paG
        sky_naF = sky_na / sky_naG
        
        #Chop差し引きした、COMQ画像に対して、Flatで割る
        comq_data = readdata(obs_comq)
        obj_obs = q_subch(comq_data)
        
        #Flatで割る
        obj_obs_datP0 = obj_obs / sky_naF
        obj_obs_datN0 = obj_obs / sky_paF    
        
        makefits(obj_obs_datP0, "./out/obj"+
                str(obsfilename[-10:-5]) + "_datP0.fits")
        makefits(obj_obs_datN0, "./out/obj" +
                str(obsfilename[-10:-5]) + "_datN0.fits")


#### フォルダの整理
ダウンロードしてきたファイルは、ダークと天体画像が混ざっている。
そこで、いったん、ダークだけを取り出し別のフォルダに移すことを考える。
FRAME ID のリストからDATA_TYPあるいは、OBJECT2がDARKであればダークと判断する。
名前が一致するものをファイルの中から探し、見つかればファイルをダークフォルダに移動する。

2009-01-13のデータで練習してみる。

'D:\all-01-13\P03yushi1102114609NN'

In [108]:
folderpath = r'D:\2009-01-13'
judgedarkfile = r'../DataFrame/COM_2009.xlsx'


In [109]:
frameall = pd.read_excel(judgedarkfile)
#1行目は不要なので削除
frameall = frameall.dropna(how="all",axis=0)
darkframe= frameall[(frameall["DATA_TYP"] =="DARK") | (frameall["OBJECT2"]=="DARK")]
#indexをリセット
darkframe = darkframe.reset_index(drop=True)

In [110]:
#darkframeのうち、日付で1/13を選択
darkframe_09_0113 = darkframe[darkframe["DATE_OBS"]=="2009-01-13"]
# darkframe_09_0113

ここまででダークは取ってこられたので、該当するものをダークフォルダを作成し移動していく

In [113]:
#ダークフォルダを作成
try:
    os.mkdir(r"D:\2009-01-13\DARK")
except FileExistsError:
    pass

#あるいは
os.makedirs(r"D:\2009-01-13\DARK",exist_ok=True)

#filepath
filepath = r"D:\2009-01-13"
filelist = glob.glob(filepath+"\*")

#darkの名前を取得
for dark in darkframe_09_0113["#FRAME_ID"]:
    #filename作成
    name = filepath +"\\"+ str(dark) +".fits"
    #移動
    if name in filelist:
        shutil.move(name, filepath+"\\DARK\\" + str(dark) +".fits")

In [114]:
darkfilepath = r"D:\2009-01-13\DARK"
darkfilelist = glob.glob(darkfilepath+"\\*")
d1 = darkfilelist[0]

In [115]:
d1head = readheader(d1)
d1head["FRAMEID"]
d1head["DATE-OBS"]
d1head["Q_PIXTIM"]
d1head["Q_RRSTRT"]
d1head["Q_YSTRT"]
d1head["Q_CHAM"]


1

In [128]:
#天体画像特定のためのDARKの情報を入れたデータフレームを作成する。
darkdf = pd.DataFrame(columns=["FRRAME_ID", "DATE", "PIXTIM", "RRSTRT", "YSTRT", "CoAdd"])
for darkfile in darkfilelist[:2]:
    darkheader = readheader(darkfile)
    # darkdf.iloc[len(darkdf)] = [darkheader["FRAMEID"], darkheader["DATE-OBS"], darkheader["Q_PIXTIM"], darkheader["Q_RRSTRT"], darkheader["Q_YSTRT"], darkheader["Q_CHAM"]]
    # [darkheader["FRAMEID"], darkheader["DATE-OBS"], darkheader["Q_PIXTIM"],
    #     darkheader["Q_RRSTRT"], darkheader["Q_YSTRT"], darkheader["Q_CHAM"]]
list1 = [darkheader["FRAMEID"], darkheader["DATE-OBS"], darkheader["Q_PIXTIM"],
        darkheader["Q_RRSTRT"], darkheader["Q_YSTRT"], darkheader["Q_CHAM"]]
add_row = pd.DataFrame(list1, columns = darkdf.columns).T
add_row 

ValueError: Shape of passed values is (6, 1), indices imply (6, 6)

In [27]:
list1 =["aoeek", "ksdnf", 1,2, 3,4]
import pandas as pd
df = pd.DataFrame(columns=["FRRAME_ID", "DATE",
                  "PIXTIM", "RRSTRT", "YSTRT", "CoAdd"])
a  = pd.DataFrame(list1, c).T
df2 = pd.concat([df, a],axis =0, ignore_index=True)
df2

,FRRAME_ID,DATE,PIXTIM,RRSTRT,YSTRT,CoAdd,0,1,2,3,4,5
0,NaN,NaN,NaN,NaN,NaN,NaN,aoeek,ksdnf,1,2,3,4


In [25]:
import numpy as np
import pandas as pd
a = pd.DataFrame([1,2,3,4,5]).T
a

,0,1,2,3,4
0,1,2,3,4,5
